In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.append('..')
from utils import preprocess, get_windows

F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
MIN_COUNTS = 2
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [4]:
nlp = spacy.load('en')

# Load dataset

In [30]:
from utils import preprocess, get_windows

In [35]:
#data_home='D:/code python/LDA2vec/bangla news',
dataset = fetch_20newsgroups(data_home='D:/code python/LDA2vec/bangla news',subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [36]:
# number of documents
len(docs)

9

In [33]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

In [9]:
print(docs[0])

(0, 'সিডনিতে ইংল্যান্ড-আফগানিস্তান ম্যাচের ধারাভাষ্যকার তাঁরা। বৃষ্টির কারণে বন্ধ থাকায় হয়তো সময় কাটাতেই টেলিভিশনে চোখ রেখেছিলেন হ্যামিল্টনের বাংলাদেশ-নিউজিল্যান্ড ম্যাচের দিকে। কিন্তু এই ম্যাচের বাংলাদেশকে দেখে বিস্ময়ে অভিভূত স্টার স্পোর্টসের তিন ধারাভাষ্যকার-রমিজ রাজা, ওয়াসিম আকরাম ও ইয়ান বিশপ। এবারের বিশ্বকাপের বাংলাদেশ দলকে দেখে সত্যিই অবাক এই সাবেক তিন তারকা। বিশপের খুব মনে আছে গত আগস্টে ক্যারিবীয় দ্বীপপুঞ্জে খেলতে যাওয়া বাংলাদেশ দলটির কথা। হারতে হারতে ক্লান্ত, ত্রস্ত-বিধ্বস্ত ওই বাংলাদেশ দলের সঙ্গে বিশ্বকাপের বাংলাদেশ দলটির কোনো মিলই খুঁজে পাচ্ছেন না ওয়েস্ট ইন্ডিজের সাবেক এই ফাস্ট বোলার। ইংল্যান্ড-আফগানিস্তান ম্যাচের এক ফাঁকে একটা আলোচনায় বাংলাদেশ প্রসঙ্গে বিশপ বললেন, ‘এই বাংলাদেশের শরীরী ভাষাই অন্য রকম। গত বছরের মাঝামাঝি সময় ওয়েস্ট ইন্ডিজ সফরে যাওয়া দলটির আত্মবিশ্বাসের সঙ্গে এই দলের কোনো কিছুরই মিল নেই।’ রমিজ রাজা বলেন, ‘২০১২ এশিয়া কাপে ভারত-শ্রীলঙ্কাকে হারিয়ে ফাইনালে গিয়েই বাংলাদেশ নিজেদের চেনাতে পেরেছিল। তারা তখনই সবাইকে জানিয়ে দিয়েছিল, নিজেদের দিনে যেকোনো দলকে হারা

# Preprocess dataset and create windows

In [16]:
from utils import preprocess, get_windows

In [34]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

['সিডনিতে', 'ইংল্যান্ড', 'আফগানিস্তান', 'ম্যাচের', 'ধারাভাষ্যকার', 'বৃষ্টির', 'বন্ধ', 'থাকায়', 'হয়তো', 'সময়', 'কাটাতেই', 'টেলিভিশনে', 'চোখ', 'রেখেছিলেন', 'হ্যামিল্টনের', 'বাংলাদেশ', 'নিউজিল্যান্ড', 'ম্যাচের', 'ম্যাচের', 'বাংলাদেশকে', 'বিস্ময়ে', 'অভিভূত', 'স্টার', 'স্পোর্টসের', 'তিন', 'ধারাভাষ্যকার', 'রমিজ', 'রাজা', 'ওয়াসিম', 'আকরাম', 'ইয়ান', 'বিশপ', 'এবারের', 'বিশ্বকাপের', 'বাংলাদেশ', 'দলকে', 'সত্যিই', 'অবাক', 'সাবেক', 'তিন', 'তারকা', 'বিশপের', 'আগস্টে', 'ক্যারিবীয়', 'দ্বীপপুঞ্জে', 'খেলতে', 'বাংলাদেশ', 'দলটির', 'কথা', 'হারতে', 'হারতে', 'ক্লান্ত', 'ত্রস্ত', 'বিধ্বস্ত', 'বাংলাদেশ', 'দলের', 'বিশ্বকাপের', 'বাংলাদেশ', 'দলটির', 'কোনো', 'মিলই', 'খুঁজে', 'পাচ্ছেন', 'ওয়েস্ট', 'ইন্ডিজের', 'সাবেক', 'ফাস্ট', 'বোলার', 'ইংল্যান্ড', 'আফগানিস্তান', 'ম্যাচের', 'ফাঁকে', 'একটা', 'আলোচনায়', 'বাংলাদেশ', 'প্রসঙ্গে', 'বিশপ', 'বাংলাদেশের', 'শরীরী', 'ভাষাই', 'বছরের', 'মাঝামাঝি', 'সময়', 'ওয়েস্ট', 'ইন্ডিজ', 'সফরে', 'দলটির', 'আত্মবিশ্বাসের', 'দলের', 'কোনো', 'কিছুরই', 'মিল', 'নেই।’', 'রমিজ', 'রাজা', '২০১২

 12%|██████████▌                                                                         | 1/8 [00:00<00:02,  3.37it/s]

['নিউজিল্যান্ডের', 'বিপক্ষে', 'ম্যাচটি', 'আত্মবিশ্বাস', 'বাড়িয়ে', 'নেওয়ার', 'হয়েছে', 'গ্রুপ', 'পর্বের', 'হিসাব', 'নিকাশ', 'শেষে', 'বাংলাদেশের', 'ভারত', 'মাশরাফি', 'বিন', 'মুর্তজার', 'অধিনায়কত্ব', 'সাকিব', 'হাসান', 'জানালেন', 'আত্মবিশ্বাস', 'লাগিয়ে', 'শেষ', 'আটের', 'লড়াইয়ে', 'দারুণ', 'চায়', 'বাংলাদেশ', 'শ্রীলঙ্কা', 'অস্ট্রেলিয়া', 'ইংল্যান্ড', 'আফগানিস্তান', 'স্কটল্যান্ড', 'দলই', 'নাকানিচুবানি', 'খেয়েছে', 'কিউইদের', 'দলকেই', 'অলআউট', 'ব্রেন্ডন', 'ম্যাককালামের', 'বিশ্বকাপে', 'একমাত্র', 'বাংলাদেশকেই', 'অলআউট', 'পারেনি', 'নিউজিল্যান্ড', 'ফেব্রুয়ারি', 'অকল্যান্ডে', 'অস্ট্রেলিয়ার', 'বিপক্ষে', 'ম্যাচটির', 'বাংলাদেশকে', 'হারাতে', 'ঘাম', 'ছুটে', 'ভিভ', 'রিচার্ডস', 'ইয়ান', 'বোথাম', 'রিকি', 'পন্টিংদের', 'চোখে', 'এবারের', 'সম্ভাব্য', 'চ্যাম্পিয়ন', 'নিউজিল্যান্ডের', 'ম্যাচ', 'শেষে', 'সাকিবও', 'সামনেও', 'আত্মবিশ্বাস', 'রাখব', 'যেভাবে', 'খেললাম', 'দারুণ', 'কোয়ার্টার', 'ফাইনালে', 'ভীষণ', 'সহায়তা', 'এখনো', 'কোয়ার্টার', 'ফাইনাল', 'ভাবিনি', 'ম্যাচটা', 'দুর্দান্তই', 'জানি', 'সেদিন', 'গ্যালারিতে', 'প্র

 25%|█████████████████████                                                               | 2/8 [00:00<00:02,  2.84it/s]

...

['আধুনিক', 'জীবনে', 'তথ্যের', 'সন্ধানে', 'রাতের', 'অনেকটা', 'সময়', 'কাটে', 'অনলাইনে', 'কম্পিউটার', 'মোবাইল', 'যন্ত্রে', 'বেশির', 'ভাগ', 'মানুষই', 'তথ্যানুসন্ধানের', 'কাজটা', 'গুগলের', 'সময়', 'ব্যক্তিগত', 'বিষয়ে', 'অনুসন্ধান', 'চালানোর', 'গুগলের', 'তথ্য', 'খোঁজার', 'প্রতিটি', 'বিষয়', 'সংরক্ষিত', 'এটার', 'চিহ্ন', 'তথ্য', 'খোঁজার', 'বিব্রতকর', 'কম্পিউটার', 'মোবাইলে', 'গুগলের', 'যেসব', 'তথ্য', 'খোঁজা', 'না-জানুক', 'তারও', 'ব্যবস্থা', 'গুগল', 'অনুসন্ধান', 'সেবার', 'হিস্ট্রি', 'ইতিহাস', 'মুছে', 'ফেলে', 'কাজটা', 'কাজটা', 'চাইলে', 'গুগল', 'অ্যাকাউন্টে', 'প্রবেশ', 'করুন', 'গুগল', 'অনুসন্ধানের', 'হিস্ট্রি', 'পাতায়', 'https://history.google.com/history', 'ওয়েব', 'ঠিকানায়', 'গুগলে', 'লগইন', 'থাকলেও', 'পাতায়', 'পাসওয়ার্ড', 'গুগলে', 'তথ্য', 'অনুসন্ধানের', 'হিস্ট্রি', 'ইতিহাস', 'পাতার', 'ওপরের', 'তথ্য', 'খোঁজার', 'প্রবণতাবিষয়ক', 'লেখচিত্র', 'নিচেই', 'remove', 'item', 'নামে', 'বোতাম', 'বাঁয়ে', 'ঘরে', 'টিক', 'বোতামটিতে', 'ক্লিক', 'করুন', 'কতটি', 'তথ্য', 'মোছা', 'বার্তা', 'চলতি', 'হিস্ট্রি', 

 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [00:02<00:00,  3.29it/s]

['জনপ্রিয়', 'প্রযুক্তিপণ্য', 'নির্মাতা', 'প্রতিষ্ঠান', 'অ্যাপলের', 'অ্যাপ', 'স্টোর', 'আইটিউনস', 'ব্যবহারের', 'সমস্যায়', 'পড়েছেন', 'গ্রাহকেরা', 'লাখো', 'অ্যাপল', 'ব্যবহারকারী', 'অ্যাপ', 'স্টোর', 'আইটিউনস', 'সেবা', 'ব্যাহত', 'সেবাগুলো', 'ব্যবহারের', 'সমস্যায়', 'পড়েছেন', 'অ্যাপ', 'স্টোর', 'অ্যাপ', 'নামানো', 'আইটিউনসের', 'গান', 'নামানোর', 'সমস্যায়', 'পড়তে', 'গ্রাহকদের', 'সামাজিক', 'যোগাযোগ', 'তুমুল', 'সমালোচনার', 'মুখে', 'পড়তে', 'অ্যাপলকে', 'এসব', 'সেবার', 'পাশাপাশি', 'আইক্লাউডে', 'প্রবেশের', 'ক্ষেত্রেও', 'সমস্যায়', 'পড়তে', 'ব্যবহারকারীদের', 'সমস্যার', 'অ্যাপল', 'অ্যাপ', 'স্টোর', 'আইটিউনস', 'লাখ', 'ডলারের', 'ক্ষতির', 'মুখে', 'পড়েছে', 'পুরো', 'সেবা', 'ব্যাহত', 'ব্যবহারকারীরা', 'অ্যাপ', 'নামানো', 'ইনস্টল', 'গান', 'ভিডিও', 'কেনা', 'পারেননি', 'সমস্যাটি', 'যুক্তরাজ্যের', 'সময়', 'সকাল', 'নয়টায়', 'সেবা', 'ব্যাহত', 'ঘটনায়', 'গ্রাহকদের', 'ক্ষমা', 'চেয়েছে', 'অ্যাপল', 'কর্তৃপক্ষ', 'আনুষ্ঠানিকভাবে', 'বার্তায়', 'অ্যাপল', 'আন্ত', 'ডোমেইন', 'নেম', 'সিস্টেম', 'ডিএনএস', 'সেবার', 'সমস্যার',

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.20it/s]


number of removed short documents: 0
total number of tokens: 1251
number of tokens to be removed: 633
number of additionally removed short documents: 1
total number of tokens: 611

minimum word count number: 2
this number can be less than MIN_COUNTS because of document removal


In [9]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

['সিডনিতে', 'ইংল্যান্ড', 'আফগানিস্তান', 'ম্যাচের', 'ধারাভাষ্যকার', 'তাঁরা', 'বৃষ্টির', 'কারণে', 'বন্ধ', 'থাকায়', 'হয়তো', 'সময়', 'কাটাতেই', 'টেলিভিশনে', 'চোখ', 'রেখেছিলেন', 'হ্যামিল্টনের', 'বাংলাদেশ', 'নিউজিল্যান্ড', 'ম্যাচের', 'দিকে', 'কিন্তু', 'ম্যাচের', 'বাংলাদেশকে', 'দেখে', 'বিস্ময়ে', 'অভিভূত', 'স্টার', 'স্পোর্টসের', 'তিন', 'ধারাভাষ্যকার', 'রমিজ', 'রাজা', 'ওয়াসিম', 'আকরাম', 'ইয়ান', 'বিশপ', 'এবারের', 'বিশ্বকাপের', 'বাংলাদেশ', 'দলকে', 'দেখে', 'সত্যিই', 'অবাক', 'সাবেক', 'তিন', 'তারকা', 'বিশপের', 'খুব', 'মনে', 'আছে', 'আগস্টে', 'ক্যারিবীয়', 'দ্বীপপুঞ্জে', 'খেলতে', 'যাওয়া', 'বাংলাদেশ', 'দলটির', 'কথা', 'হারতে', 'হারতে', 'ক্লান্ত', 'ত্রস্ত', 'বিধ্বস্ত', 'বাংলাদেশ', 'দলের', 'সঙ্গে', 'বিশ্বকাপের', 'বাংলাদেশ', 'দলটির', 'কোনো', 'মিলই', 'খুঁজে', 'পাচ্ছেন', 'ওয়েস্ট', 'ইন্ডিজের', 'সাবেক', 'ফাস্ট', 'বোলার', 'ইংল্যান্ড', 'আফগানিস্তান', 'ম্যাচের', 'ফাঁকে', 'একটা', 'আলোচনায়', 'বাংলাদেশ', 'প্রসঙ্গে', 'বিশপ', 'বললেন', 'বাংলাদেশের', 'শরীরী', 'ভাষাই', 'অন্য', 'রকম', 'বছরের', 'মাঝামাঝি', 'সময়', 'ওয়ে

 12%|██████████▌                                                                         | 1/8 [00:00<00:02,  2.91it/s]

['নিউজিল্যান্ডের', 'বিপক্ষে', 'ম্যাচটি', 'ছিল', 'আত্মবিশ্বাস', 'বাড়িয়ে', 'নেওয়ার', 'সেটি', 'হয়েছে', 'গ্রুপ', 'পর্বের', 'হিসাব', 'নিকাশ', 'শেষে', 'বাংলাদেশের', 'সামনে', 'এখন', 'ভারত', 'মাশরাফি', 'বিন', 'মুর্তজার', 'বদলে', 'অধিনায়কত্ব', 'করা', 'সাকিব', 'হাসান', 'জানালেন', 'আত্মবিশ্বাস', 'কাজে', 'লাগিয়ে', 'শেষ', 'আটের', 'লড়াইয়ে', 'দারুণ', 'কিছুই', 'করতে', 'চায়', 'বাংলাদেশ', 'শ্রীলঙ্কা', 'অস্ট্রেলিয়া', 'ইংল্যান্ড', 'আফগানিস্তান', 'স্কটল্যান্ড', 'দলই', 'নাকানিচুবানি', 'খেয়েছে', 'কিউইদের', 'কাছে', 'দলকেই', 'অলআউট', 'করেছে', 'ব্রেন্ডন', 'ম্যাককালামের', 'বিশ্বকাপে', 'একমাত্র', 'বাংলাদেশকেই', 'অলআউট', 'করতে', 'পারেনি', 'নিউজিল্যান্ড', 'ফেব্রুয়ারি', 'অকল্যান্ডে', 'অস্ট্রেলিয়ার', 'বিপক্ষে', 'ম্যাচটির', 'বাংলাদেশকে', 'হারাতে', 'ঘাম', 'ছুটে', 'গেছে', 'ভিভ', 'রিচার্ডস', 'ইয়ান', 'বোথাম', 'রিকি', 'পন্টিংদের', 'চোখে', 'এবারের', 'সম্ভাব্য', 'চ্যাম্পিয়ন', 'নিউজিল্যান্ডের', 'ম্যাচ', 'শেষে', 'সাকিবও', 'বললেন', 'অবশ্যই', 'সামনেও', 'আমরা', 'আত্মবিশ্বাস', 'ধরে', 'রাখব', 'যেভাবে', 'খেললাম', 'সেটা', 'দারুণ', 

 25%|█████████████████████                                                               | 2/8 [00:00<00:02,  2.33it/s]

...

['আধুনিক', 'জীবনে', 'বিভিন্ন', 'তথ্যের', 'সন্ধানে', 'দিন', 'রাতের', 'অনেকটা', 'সময়', 'কাটে', 'অনলাইনে', 'হোক', 'সেটা', 'কম্পিউটার', 'মোবাইল', 'যন্ত্রে', 'বেশির', 'ভাগ', 'মানুষই', 'তথ্যানুসন্ধানের', 'কাজটা', 'করেন', 'গুগলের', 'মাধ্যমে', 'অনেক', 'সময়', 'ব্যক্তিগত', 'কিছু', 'বিষয়ে', 'অনুসন্ধান', 'চালানোর', 'সেটা', 'যেন', 'অন্য', 'কেউ', 'দেখতে', 'পারেন', 'সেটা', 'চান', 'অনেকেই', 'যদিও', 'গুগলের', 'মাধ্যমে', 'তথ্য', 'খোঁজার', 'প্রতিটি', 'বিষয়', 'সংরক্ষিত', 'থাকে', 'এবং', 'এটার', 'চিহ্ন', 'থেকেই', 'যায়', 'তথ্য', 'খোঁজার', 'বিষয়টি', 'যদি', 'বিব্রতকর', 'মনে', 'হয়', 'যদি', 'চান', 'নিজের', 'কম্পিউটার', 'মোবাইলে', 'গুগলের', 'মাধ্যমে', 'যেসব', 'তথ্য', 'খোঁজা', 'হয়েছে', 'সেটা', 'কেউ', 'না-জানুক', 'তাহলে', 'তারও', 'ব্যবস্থা', 'আছে', 'ক্ষেত্রে', 'নিজের', 'গুগল', 'অনুসন্ধান', 'সেবার', 'হিস্ট্রি', 'ইতিহাস', 'মুছে', 'ফেলে', 'কাজটা', 'করা', 'যায়', 'কাজটা', 'করতে', 'চাইলে', 'নিজের', 'গুগল', 'অ্যাকাউন্টে', 'প্রবেশ', 'করুন', 'তারপর', 'গুগল', 'অনুসন্ধানের', 'হিস্ট্রি', 'পাতায়', 'যেতে', 'হলে', 'http

 88%|█████████████████████████████████████████████████████████████████████████▌          | 7/8 [00:02<00:00,  3.06it/s]

['জনপ্রিয়', 'প্রযুক্তিপণ্য', 'নির্মাতা', 'প্রতিষ্ঠান', 'অ্যাপলের', 'অ্যাপ', 'স্টোর', 'এবং', 'আইটিউনস', 'ব্যবহারের', 'ক্ষেত্রে', 'সমস্যায়', 'পড়েছেন', 'গ্রাহকেরা', 'লাখো', 'অ্যাপল', 'ব্যবহারকারী', 'অ্যাপ', 'স্টোর', 'এবং', 'আইটিউনস', 'সেবা', 'ব্যাহত', 'হওয়ার', 'ফলে', 'সেবাগুলো', 'ব্যবহারের', 'ক্ষেত্রে', 'সমস্যায়', 'পড়েছেন', 'অ্যাপ', 'স্টোর', 'থেকে', 'অ্যাপ', 'নামানো', 'কিংবা', 'আইটিউনসের', 'মাধ্যমে', 'গান', 'নামানোর', 'ক্ষেত্রে', 'সমস্যায়', 'পড়তে', 'হয়েছে', 'গ্রাহকদের', 'নিয়ে', 'সামাজিক', 'যোগাযোগ', 'মাধ্যমে', 'তুমুল', 'সমালোচনার', 'মুখে', 'পড়তে', 'হয়েছে', 'অ্যাপলকে', 'এসব', 'সেবার', 'পাশাপাশি', 'আইক্লাউডে', 'প্রবেশের', 'ক্ষেত্রেও', 'সমস্যায়', 'পড়তে', 'হয়েছে', 'ব্যবহারকারীদের', 'সমস্যার', 'ফলে', 'অ্যাপল', 'অ্যাপ', 'স্টোর', 'এবং', 'আইটিউনস', 'প্রায়', 'লাখ', 'ডলারের', 'ক্ষতির', 'মুখে', 'পড়েছে', 'বলে', 'জানা', 'গেছে', 'পুরো', 'সেবা', 'ব্যাহত', 'হওয়ার', 'ফলে', 'ব্যবহারকারীরা', 'নতুন', 'অ্যাপ', 'নামানো', 'ইনস্টল', 'করা', 'গান', 'কিংবা', 'ভিডিও', 'কেনা', 'ইত্যাদি', 'করতে', 'পা

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.06it/s]


number of removed short documents: 0
total number of tokens: 1689
number of tokens to be removed: 707
number of additionally removed short documents: 1
total number of tokens: 969

minimum word count number: 2
this number can be less than MIN_COUNTS because of document removal


In [16]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [17]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

7it [00:00, 448.21it/s]


In [18]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [19]:
# number of windows (equals to the total number of tokens)
data.shape[0]

611

In [20]:
decoder

{0: 'বাংলাদেশ',
 1: 'অ্যাপ',
 2: 'বিশ্বকাপে',
 3: 'তথ্য',
 4: 'ক্লিক',
 5: 'স্টোর',
 6: 'কোয়ার্টার',
 7: 'সাকিব',
 8: 'রান',
 9: 'হিস্ট্রি',
 10: 'করুন',
 11: 'ম্যাচের',
 12: 'বিপক্ষে',
 13: 'আফগানদের',
 14: 'সেঞ্চুরি',
 15: 'মাহমুদউল্লাহ',
 16: 'টানা',
 17: 'আইটিউনস',
 18: 'সমস্যায়',
 19: 'ব্যাহত',
 20: 'ইংল্যান্ড',
 21: 'আফগানিস্তান',
 22: 'ফাইনালে',
 23: 'কাজটা',
 24: 'বছর',
 25: 'লাখ',
 26: 'সাঙ্গাকারা',
 27: 'remove',
 28: 'অ্যাপল',
 29: 'সেবা',
 30: 'ইয়ান',
 31: 'বিশ্বকাপের',
 32: 'দলের',
 33: 'বাংলাদেশের',
 34: 'হিসাব',
 35: 'হয়ে',
 36: 'উইকেট',
 37: 'প্রাণী',
 38: 'মাহমুদউল্লাহর',
 39: 'সময়',
 40: 'গুগল',
 41: 'পড়তে',
 42: 'ডলার',
 43: 'বন্ধ',
 44: 'তিন',
 45: 'রমিজ',
 46: 'রাজা',
 47: 'বিশপ',
 48: 'খেলতে',
 49: 'দলটির',
 50: 'নিউজিল্যান্ডের',
 51: 'আত্মবিশ্বাস',
 52: 'শেষ',
 53: 'দারুণ',
 54: 'স্কটল্যান্ড',
 55: 'একমাত্র',
 56: 'মাঠে',
 57: 'রানের',
 58: 'এএফপি',
 59: 'ইংলিশরা',
 60: 'মুখে',
 61: 'ব্যাট',
 62: 'দশমিক',
 63: 'খেলা',
 64: 'নামে',
 65: 'ওভারে',
 66: 'সর্বোচ্চ

# Get unigram distribution

In [21]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

In [22]:
word_counts

array([15, 10,  8,  8,  8,  8,  7,  7,  7,  7,  7,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2])

# Prepare word vectors

In [23]:
%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

KeyError: "word '30' not in vocabulary"

In [24]:
# number of unique words
vocab_size

204

# Prepare initialization for document weights

In [25]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [26]:
%%time
n_topics = 2
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

Wall time: 484 ms


In [28]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : অ্যাপ স্টোর আইটিউনস সমস্যায় ব্যাহত সেবা অ্যাপল লাখ বাংলাদেশ পড়তে
topic 1 : বাংলাদেশ বিশ্বকাপে তথ্য ক্লিক রান সেঞ্চুরি কোয়ার্টার সাকিব ম্যাচের হিস্ট্রি


In [29]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 447.97it/s]


# Save data

In [ ]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)